<a href="https://colab.research.google.com/github/fcela/chr/blob/master/CHR_extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uncomment the line below if you want to install additional R dependencies
# needed for BART and Quantile Regression Forest
#
#!R -e "install.packages('BART', 'quantregForest')"
!pip install git+https://github.com/fcela/scikit-garden
!pip install git+https://github.com/fcela/chr
!pip install lightgbm

  Cloning https://github.com/fcela/scikit-garden to /tmp/pip-req-build-xhn3s5qg
  Running command git clone -q https://github.com/fcela/scikit-garden /tmp/pip-req-build-xhn3s5qg
  Created wheel for scikit-garden: filename=scikit_garden-0.1.3-cp37-cp37m-linux_x86_64.whl size=1006363 sha256=c158d96dc2884eaeeb5df3b70a2fb8d7e0d1efea9157593ae5f0cef47e595dea
  Stored in directory: /tmp/pip-ephem-wheel-cache-stf4a_1u/wheels/08/84/9a/f9188194893215c75af7e551fd6152642c93944bae0146f1c4
Successfully built scikit-garden
  Cloning https://github.com/fcela/chr to /tmp/pip-req-build-ki5fgl_k
  Running command git clone -q https://github.com/fcela/chr /tmp/pip-req-build-ki5fgl_k
  Created wheel for chr: filename=chr-0.0.1.dev18-py3-none-any.whl size=18830 sha256=f06bedda6c51b33143c6b58e8b295a41fd88d792ee78d0f8eb8bdbb69926ea85
  Stored in directory: /tmp/pip-ephem-wheel-cache-l95xkny5/wheels/3b/cd/41/ddf95531a0aa539ed81ad7e53a9b00e54eb2655cc80d6d673d
Successfully built chr


In [2]:
import os
import sys
import pdb
import torch

print("Is CUDA available? {}".format(torch.cuda.is_available()))

Is CUDA available? True


In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split

## Split-conformal calibration

Split-conformal with pre-trained black box

In [4]:
import pandas as pd
data = pd.read_csv("https://drive.google.com/u/0/uc?id=1BV2kV0b5O9u2E6dg8nJ-fXImH7wEcIV3&export=download")
data

,Median_House_Value,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco
0,452600.0,8.3252,41,880,129,322,126,37.88,-122.23,9263.040773,556529.158342,735501.806984,67432.517001,21250.213767
1,358500.0,8.3014,21,7099,1106,2401,1138,37.86,-122.22,10225.733072,554279.850069,733236.884360,65049.908574,20880.600400
2,352100.0,7.2574,52,1467,190,496,177,37.85,-122.24,8259.085109,554610.717069,733525.682937,64867.289833,18811.487450
3,341300.0,5.6431,52,1274,235,558,219,37.85,-122.25,7768.086571,555194.266086,734095.290744,65287.138412,18031.047568
4,342200.0,3.8462,52,1627,280,565,259,37.85,-122.25,7768.086571,555194.266086,734095.290744,65287.138412,18031.047568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,78100.0,1.5603,25,1665,374,845,330,39.48,-121.09,162031.481121,654530.186299,830631.543047,248510.058162,222619.890417
20636,77100.0,2.5568,18,697,150,356,114,39.49,-121.21,160445.433537,659747.068444,836245.915229,246849.888948,218314.424634
20637,92300.0,1.7000,17,2254,485,1007,433,39.43,-121.22,153754.341182,654042.214020,830699.573163,240172.220489,212097.936232
20638,84700.0,1.8672,18,1860,409,741,349,39.43,-121.32,152005.022239,657698.007703,834672.461887,238193.865909,207923.199166


In [5]:
Y_data = data.Median_House_Value.values
X_data = data.copy()
X_data = X_data.drop(columns="Median_House_Value").to_numpy()

In [6]:
import numpy as np
X_data_mu = np.mean(X_data, axis=0)
X_data_sd = np.std(X_data, axis=0)
X_data_normalized = (X_data - X_data_mu) / X_data_sd

Y_data_mu = np.mean(Y_data)
Y_data_sd = np.std(Y_data)
Y_data_normalized = (Y_data - Y_data_mu) / Y_data_sd


## Fit an estimator for the quantiles

QNet will fit a neural network to estimate quantiles, with the following architecture:

```
nn.Sequential(
    nn.Linear(num_features, num_hidden),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(num_hidden, num_hidden),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(num_hidden, num_quantiles),
)
```

(Expand code below for details)



In [7]:
#@title
# This is the base implementation of the methods in the original paper, for
# ease of access and to enable modifications as needed


import torch
import torch.nn as nn
import torch.optim as optim
#import torch.tensor as tensor
from torch import Tensor as tensor
from torch.utils.data import DataLoader

import six
import sys
sys.modules['sklearn.externals.six'] = six

from skgarden import RandomForestQuantileRegressor

from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import numpy as np

from chr.utils import RegressionDataset

import warnings

import pdb

class NNet(nn.Module):
    """ Conditional quantile estimator, formulated as neural net
    """
    def __init__(self, quantiles, num_features, num_hidden=64, dropout=0.1, no_crossing=False):
        """ Initialization
        Parameters
        ----------
        quantiles : numpy array of quantile levels (q), each in the range (0,1)
        num_features : integer, input signal dimension (p)
        num_hidden : integer, hidden layer dimension
        dropout : float, dropout rate
        no_crossing: boolean, whether to explicitly prevent quantile crossovers
        """
        super(NNet, self).__init__()

        self.no_crossing = no_crossing

        self.num_quantiles = len(quantiles)

        # Construct base network
        self.base_model = nn.Sequential(
            nn.Linear(num_features, num_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(num_hidden, self.num_quantiles),
        )
        self.init_weights()

    def init_weights(self):
        """ Initialize the network parameters
        """
        for m in self.base_model:
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        """ Run forward pass
        """
        x = self.base_model(x)
        if self.no_crossing:
            y,_ = torch.sort(x,1)
        else:
            y = x
        return y



class LogCoshQuantileLoss(nn.Module):
    """ Pinball loss function
    """
    def __init__(self, quantiles):
        """ Initialize
        Parameters
        ----------
        quantiles : pytorch vector of quantile levels, each in the range (0,1)
        """
        super().__init__()
        self.quantiles = quantiles

    def forward(self, preds, target):
        """ Compute the pinball loss
        Parameters
        ----------
        preds : pytorch tensor of estimated labels (n)
        target : pytorch tensor of true labels (n)
        Returns
        -------
        loss : cost function value
        """
        #assert not target.requires_grad
        #assert preds.size(0) == target.size(0)


        Q = self.quantiles.unsqueeze(0)

        errors = torch.log(torch.cosh(target.unsqueeze(1)-preds))

        loss = torch.max((Q-1.0)*errors, Q*errors).mean()

        return loss



class AllQuantileLoss(nn.Module):
    """ Pinball loss function
    """
    def __init__(self, quantiles):
        """ Initialize
        Parameters
        ----------
        quantiles : pytorch vector of quantile levels, each in the range (0,1)
        """
        super().__init__()
        self.quantiles = quantiles

    def forward(self, preds, target):
        """ Compute the pinball loss
        Parameters
        ----------
        preds : pytorch tensor of estimated labels (n)
        target : pytorch tensor of true labels (n)
        Returns
        -------
        loss : cost function value
        """
        #assert not target.requires_grad
        #assert preds.size(0) == target.size(0)

        errors = target.unsqueeze(1)-preds
        Q = self.quantiles.unsqueeze(0)
        loss = torch.max((Q-1.0)*errors, Q*errors).mean()

        return loss


class PenalizedAllQuantileLoss(nn.Module):
    """ Pinball loss function
    """
    def __init__(self, quantiles, penalty=0.01):
        """ Initialize
        Parameters
        ----------
        quantiles : pytorch vector of quantile levels, each in the range (0,1)
        """
        super().__init__()
        self.quantiles = quantiles
        self.penalty = penalty

    def forward(self, preds, target):
        """ Compute the pinball loss
        Parameters
        ----------
        preds : pytorch tensor of estimated labels (n)
        target : pytorch tensor of true labels (n)
        Returns
        -------
        loss : cost function value
        """
        #assert not target.requires_grad
        #assert preds.size(0) == target.size(0)

        violations = torch.where(torch.diff(preds, dim=0)<0, 1.0, 0.0)

        errors = target.unsqueeze(1)-preds
        Q = self.quantiles.unsqueeze(0)
        loss = torch.max((Q-1.0)*errors, Q*errors).mean() + \
                violations.mean() * self.penalty
                
        #print(f"DEBUG> preds.shape={preds.shape}, target.shape={target.shape}")
        #print(f"DEBUG> violations.shape={violations.shape}")
        #print(f"DEBUG> pred={preds}")
        #print(f"DEBUG> violations={violations}")
        #print(f"DEBUG> base_loss = {torch.max((Q-1.0)*errors, Q*errors).mean()}")
        #print(f"DEBUG> violations_loss = {violations.mean() * self.penalty}")
        return loss


class QNet:
    """ Fit a neural network (conditional quantile) to training data
    """
    def __init__(self, quantiles, num_features, no_crossing=False, dropout=0.2, learning_rate=0.001,
                 num_epochs=100, batch_size=16, num_hidden=64, random_state=0, calibrate=0, 
                 use_logcosh = False, rank_order_penalty = 0,
                 verbose=False):
        """ Initialization
        Parameters
        ----------
        quantiles : numpy array of quantile levels (q), each in the range (0,1)
        num_features : integer, input signal dimension (p)
        learning_rate : learning rate
        random_state : integer, seed used in CV when splitting to train-test
        """

        # Detect whether CUDA is available
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Store input (sort the quantiles)
        quantiles = np.sort(quantiles)
        self.quantiles = torch.from_numpy(quantiles).float().to(self.device)
        self.num_features = num_features

        # Define NNet model
        self.model = NNet(self.quantiles, self.num_features, num_hidden=num_hidden, dropout=dropout, no_crossing=no_crossing)
        self.model.to(self.device)

        # Initialize optimizer
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        # Initialize loss function
        if use_logcosh:
          self.loss_func = LogCoshQuantileLoss(self.quantiles)
        elif rank_order_penalty >0:
          self.loss_func = PenalizedAllQuantileLoss(self.quantiles, rank_order_penalty)
        else:
          self.loss_func = AllQuantileLoss(self.quantiles)

        # Store variables
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.random_state = random_state
        self.calibrate = int(calibrate)

        # Initialize training logs
        self.loss_history = []
        self.test_loss_history = []
        self.full_loss_history = []

        # Validation
        self.val_period = 10

        self.verbose = verbose

    def fit(self, X, Y, return_loss=False):
        Y = Y.flatten().astype(np.float32)
        X = X.astype(np.float32)
        
        dataset = RegressionDataset(X, Y)
        num_epochs = self.num_epochs
        if self.calibrate>0:
            # Train with 80% of samples
            n_valid = int(np.round(0.2*X.shape[0]))
            loss_stats = []
            for b in range(self.calibrate):
                X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=n_valid, random_state=self.random_state+b)
                train_dataset = RegressionDataset(X_train, Y_train)
                val_dataset = RegressionDataset(X_valid, Y_valid)
                loss_stats_tmp = self._fit(train_dataset, num_epochs, val_dataset=val_dataset)
                loss_stats.append([loss_stats_tmp['val']])                
                # Reset model
                self.model.init_weights()

            loss_stats = np.matrix(np.concatenate(loss_stats,0)).T

            loss_stats = np.median(loss_stats,1).flatten()
            # Find optimal number of epochs
            num_epochs = self.val_period*(np.argmin(loss_stats)+1)
            loss_stats_cal = loss_stats

        # Train with all samples
        loss_stats = self._fit(dataset, num_epochs)
        if self.calibrate:
            loss_stats = loss_stats_cal

        if return_loss:
          return loss_stats

    def _fit(self, train_dataset, num_epochs, val_dataset=None):
        batch_size = self.batch_size

        # Initialize data loaders
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
        if val_dataset is not None:
            val_loader = DataLoader(dataset=val_dataset, batch_size=1)

        num_samples, num_features = train_dataset.X_data.shape
        print("Training with {} samples and {} features.". \
              format(num_samples, num_features))

        loss_stats = {'train': [], "val": []}

        X_train_batch = train_dataset.X_data.to(self.device)
        y_train_batch = train_dataset.y_data.to(self.device)

        for e in tqdm(range(1, num_epochs+1)):

            # TRAINING
            train_epoch_loss = 0
            self.model.train()

            if batch_size<500:
            
              for X_train_batch, y_train_batch in train_loader:
                  X_train_batch, y_train_batch = X_train_batch.to(self.device), y_train_batch.to(self.device)
                  self.optimizer.zero_grad()

                  y_train_pred = self.model(X_train_batch).to(self.device)

                  train_loss = self.loss_func(y_train_pred, y_train_batch)

                  train_loss.backward()
                  self.optimizer.step()

                  train_epoch_loss += train_loss.item()

            else:
                self.optimizer.zero_grad()

                y_train_pred = self.model(X_train_batch).to(self.device)

                train_loss = self.loss_func(y_train_pred, y_train_batch)

                train_loss.backward()
                self.optimizer.step()

                train_epoch_loss += train_loss.item()

            # VALIDATION
            if val_dataset is not None:
                if e % self.val_period == 0:
                    self.model.eval()
                    with torch.no_grad():
                        val_epoch_loss = 0
                        for X_val_batch, y_val_batch in val_loader:
                            X_val_batch, y_val_batch = X_val_batch.to(self.device), y_val_batch.to(self.device)
                            y_val_pred = self.model(X_val_batch).to(self.device)
                            val_loss = self.loss_func(y_val_pred, y_val_batch)
                            val_epoch_loss += val_loss.item()

                    loss_stats['val'].append(val_epoch_loss/len(val_loader))
                    self.model.train()

            else:
                loss_stats['val'].append(0)

            if e % self.val_period == 0:
                loss_stats['train'].append(train_epoch_loss/len(train_loader))

            if (e % 10 == 0) and (self.verbose):
                if val_dataset is not None:
                    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | ', end='')
                    print(f'Val Loss: {val_epoch_loss/len(val_loader):.5f} | ', flush=True)
                else:
                    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | ', flush=True)

        return loss_stats

    def predict(self, X):
        """ Estimate the label given the features
        Parameters
        ----------
        x : numpy array of training features (nXp)
        Returns
        -------
        ret_val : numpy array of predicted labels (n)
        """
        self.model.eval()
        ret_val = self.model(torch.from_numpy(X).to(self.device).float().requires_grad_(False))
        return ret_val.cpu().detach().numpy()

    def get_quantiles(self):
        return self.quantiles.cpu().numpy()
    

class QRF:
    """ Fit a random forest (conditional quantile) to training data
    """
    def __init__(self, quantiles, min_samples_leaf=5, n_estimators=100, n_jobs=1,
                 random_state=0, verbose=False):
        """ Initialization
        Parameters
        ----------
        quantiles : numpy array of quantile levels (q), each in the range (0,1)
        num_features : integer, input signal dimension (p)
        random_state : integer, seed used in quantile random forests
        """

        self.device = 'cpu'
        # Store input (sort the quantiles)
        self.quantiles = torch.from_numpy(np.sort(quantiles)).float().to(self.device) 
        # Define RF model
        self.model = RandomForestQuantileRegressor(random_state=random_state,
                                                   min_samples_leaf=min_samples_leaf,
                                                   n_estimators=n_estimators,
                                                   n_jobs=n_jobs, verbose=verbose)

    def fit(self, X, Y, return_loss=None):
        warnings.filterwarnings("ignore", category=FutureWarning)
        self.model.fit(X, Y)
        warnings.filterwarnings("default", category=FutureWarning)
        return 0

    def predict(self, X):
        """ Estimate the label given the features
        Parameters
        ----------
        x : numpy array of training features (nXp)
        Returns
        -------
        ret_val : numpy array of predicted labels (n)
        """
        quantiles = self.quantiles.cpu()
        ret_val = np.zeros((X.shape[0], len(quantiles)))
        print("Predicting RF quantiles:")
        for i in tqdm(range(len(quantiles))):
            ret_val[:,i] = self.model.predict(X,quantile=100*quantiles[i])
        return ret_val
    
    def get_quantiles(self):
        return self.quantiles.cpu().numpy()



import lightgbm as lgb


class BQR:
    """ Fit multiple quantile regressions via boosting
    """
    def __init__(self, quantiles, min_samples_leaf=5, n_estimators=100, n_jobs=1,
                 random_state=0, verbose=False, categorical_features = "auto", max_bin = 127, 
            num_leaves = 7, min_data_in_leaf = 15, learning_rate = 0.025,
            feature_fraction = 1.0, bagging_fraction = 0.8, bagging_freq = 1):
    
        """ Initialization
        Parameters
        ----------
        quantiles : numpy array of quantile levels (q), each in the range (0,1)
        random_state : integer, seed used in quantile random forests
        """

        self.device = 'cpu'
        # Store input (sort the quantiles)
        self.quantiles = torch.from_numpy(np.sort(quantiles)).float().cpu().numpy()
        self.random_state = random_state
        self.models = []
        self.categorical_features = categorical_features
        self.max_bin = max_bin
        self.num_leaves = num_leaves
        self.min_data_in_leaf = min_data_in_leaf
        self.learning_rate = learning_rate,
        self.feature_fraction = feature_fraction
        self.bagging_fraction = bagging_fraction
        self.bagging_freq = bagging_freq
        

    def fit(self, X, Y):

        train_set = lgb.Dataset(
                      data =X,
                      categorical_feature = self.categorical_features,
                      free_raw_data = False,
                      label = Y)

        for i in tqdm(range(len(self.quantiles))):
        
            model = lgb.train(
                        train_set = train_set,
                        #early_stopping_rounds = 250,
                        num_boost_round = 1000,
                        verbose_eval = 50,
                        params = {
                            'alpha': self.quantiles[i],
                            'objective':'quantile',
                            'metric':'quantile',
                            'max_bin': self.max_bin,
                            'num_leaves': self.num_leaves,
                            'min_data_in_leaf': self.min_data_in_leaf,
                            'learning_rate': self.learning_rate,
                            'feature_fraction': self.feature_fraction,
                            'bagging_fraction': self.bagging_fraction,
                            'bagging_freq': self.bagging_freq,
                            'seed': self.random_state})
          
            self.models.append(model)


    def predict(self, X, no_crossing = True):
        """ Estimate the label given the features
        Parameters
        ----------
        x : numpy array of training features (nXp)
        Returns
        -------
        ret_val : numpy array of predicted labels (n)
        """

        ret_val = np.zeros((X.shape[0], len(self.quantiles)))
        print("Predicting quantiles:")
        for i in tqdm(range(len(self.quantiles))):
            ret_val[:,i] = self.models[i].predict(X)

        if no_crossing:
           ret_val = np.sort(ret_val, axis=1)

        return ret_val
    
    def get_quantiles(self):
        return self.quantiles

In [8]:
from chr.methods import CHR

  
# Split the data
X_train, X_calib_test, Y_train, Y_calib_test = train_test_split(X_data_normalized, Y_data_normalized, test_size=0.5, random_state=2020)
X_calib, X_test, Y_calib, Y_test = train_test_split(X_calib_test, Y_calib_test, test_size=0.5, random_state=2020)
grid_quantiles = np.arange(0.025,1.0,0.025)




In [32]:

# Default to quantile regression via boosting 
method = "BQR" 

if method == "QNet":

  bbox = QNet(grid_quantiles, num_features=X_train.shape[1], no_crossing=True, batch_size=1000, dropout=0.1, 
              num_epochs=10000, learning_rate=0.0005, num_hidden=256, calibrate=0)
  
elif method == "QNet-logcosh":
  # Multiple quantile regression via a neural network, using the logcosh smoothed pinball loss

  bbox = QNet(grid_quantiles, num_features=X_train.shape[1], no_crossing=False, batch_size=1000, dropout=0.1, 
              num_epochs=10000, learning_rate=0.0005, num_hidden=256, use_logcosh=True, calibrate=0)
  
elif method == "QNet-penalty":
  # Multiple quantile regression via a neural network, using the pinball loss with a penalty on rank order violations
  bbox = QNet(grid_quantiles, num_features=X_train.shape[1], no_crossing=False, batch_size=1000, dropout=0.1, 
              num_epochs=10000, learning_rate=0.0005, num_hidden=256, rank_order_penalty=0.1, calibrate=0)


elif method == "QRF":
  bbox = QRF(grid_quantiles, n_estimators=100, min_samples_leaf=50, random_state=2020)

elif method == "BQR":
  bbox = BQR(grid_quantiles, random_state=2020)


bbox.fit(X_train, Y_train)

  0%|          | 0/39 [00:00<?, ?it/s]

In [33]:
# Initialize and calibrate the new method
chr = CHR(bbox, ymin=-3, ymax=3, y_steps=200, delta_alpha=0.001, randomize=True)
chr.calibrate(X_calib, Y_calib, alpha=0.1)

Predicting quantiles:


  0%|          | 0/39 [00:00<?, ?it/s]

Computing conformity scores.


  0%|          | 0/999 [00:00<?, ?it/s]

Calibrated alpha (nominal level: 0.1): 0.094.


0.094

In [34]:
bands = chr.predict(X_test)
bands


Predicting quantiles:


  0%|          | 0/39 [00:00<?, ?it/s]

array([[-0.70854271,  0.25628141],
       [-0.49748744,  0.82914573],
       [ 0.34673367,  2.84924623],
       ...,
       [-1.22110553, -0.76884422],
       [ 0.82914573,  2.81909548],
       [-1.28140704, -0.67839196]])

In [35]:
# Compute the inter-quantile range and check there are no crossings
iqr = bands[:,1] - bands[:,0]

pct_out_of_ci = np.mean(np.mean( (Y_test > bands[:,1]) | (Y_test < bands[:,0] ) ))
pct_crossings = np.mean(iqr<0)
avg_iqr = np.mean(iqr)

print(f"Average IQR: {avg_iqr}; {100*(1-pct_out_of_ci)}% coverage; {pct_crossings*100}% of crossed IQRs")
print("For reference, for a target 90% CI .. ")
print("BQR has an avg IQR of ~ 1.2545109267266563 and 0% crossed IQRS")
print("QNet has an avg IQR of ~ 1.18270421876826; coverage of 90.17441860465117%, and 0% crossed IQR's")

Average IQR: 1.2547855556854037; 90.52325581395348% coverage; 0.0% of crossed IQRs
For reference, for a target 90% CI .. 
BQR has an avg IQR of ~ 1.2545109267266563 and 0% crossed IQRS
QNet has an avg IQR of ~ 1.18270421876826; coverage of 90.17441860465117%, and 0% crossed IQR's


In [36]:
# Compute all the quantiles, use the median as our point prediction
q_hat = bbox.predict(X_test)

Predicting quantiles:


  0%|          | 0/39 [00:00<?, ?it/s]

In [40]:
Y_test_hat = (q_hat[:,grid_quantiles == 0.5]).flatten()

In [41]:
err = np.abs(Y_test - Y_test_hat)

In [42]:
# Show top places where we have high error and tight bounds -- i.e. we are 
# overconfident

XX = pd.DataFrame(X_test)
XX.columns = data.drop(columns="Median_House_Value").columns.values
XX['err'] = err
XX['iqr'] = iqr
XX['norm_err'] = err/iqr
XX.sort_values(by=['norm_err'], ascending=False)

,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco,err,iqr,norm_err
1387,-0.336130,-1.401600,0.815579,0.439424,0.083469,0.333386,1.441143,-0.808739,0.770795,1.258247,1.247256,-0.849248,-0.945197,2.126681e+00,0.753769,2.821397e+00
1734,5.858286,1.856182,-1.107355,-1.200980,-1.210204,-1.238594,0.729500,-1.008392,-0.050892,0.776117,0.837329,-1.463860,-1.151714,3.089838e+00,1.206030,2.561991e+00
453,-0.841716,-0.209729,-0.678304,-0.343982,0.401368,-0.260357,-0.838925,0.828409,-0.407313,-0.931012,-0.881413,0.831707,0.844147,1.801259e+00,0.814070,2.212657e+00
1270,-0.258331,-1.798890,9.848555,9.458085,2.428858,4.502663,-0.867017,1.602062,1.534628,-0.375375,-0.892661,1.328275,1.272039,1.182631e+00,0.542714,2.179107e+00
824,0.196091,-1.242684,1.606755,1.142115,0.995663,1.222692,1.361551,-1.048322,0.210621,1.277157,1.267325,-0.984738,-1.113831,1.568295e+00,0.783920,2.000581e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,-1.264292,-1.639974,-1.098187,-1.167745,-1.203139,-1.243825,-1.082382,1.791732,0.761803,-0.180500,-0.930101,1.604811,1.512976,2.514068e-06,1.296482,1.939145e-06
373,5.255846,1.856182,-0.100737,-0.586125,-0.543499,-0.600386,-0.740606,0.538914,-0.744559,-0.995968,-0.709077,0.590083,0.634053,5.774526e-08,0.361809,1.596015e-07
4872,3.092065,-0.686477,3.706626,2.561741,2.035900,2.645582,-0.707833,0.558879,-0.569387,-1.005070,-0.696166,0.573243,0.619605,5.774526e-08,0.753769,7.660871e-08
4314,1.342384,1.856182,0.526795,0.641210,-0.048106,0.783270,1.010411,-1.432653,-0.803694,1.179369,1.181439,-1.286425,-1.538281,5.774526e-08,1.688442,3.420032e-08


### Now identify slices where performance is particularly bad

TODO: using slice learning, identify interesting regions; some ideas:

* Regions where we have small normalized error in training, but very high in testing (hinting overfitting)
* Regions that have a very broad IQR (hinting high uncertainty, needed of more data)

### More things to do

* Explore alternative ways of computing the quantiles, particularly methods that can guarantee no crossings:
 * [xgboost quantile regression with "smoothed elbow" pinball loss](https://colab.research.google.com/drive/1KlRkrLi7JmVpprL94vN96lZU-HyFNkTq?usp=sharing) ... this is probably one of the better ways to do it, but introduces the annoyance of having to do hyperparameter tuning to figure out the right level of smoothing in the elbow
 * [xgboost quantile regression with logcosh loss](https://towardsdatascience.com/confidence-intervals-for-xgboost-cac2955a8fde) ... this is an interesting hack to come up with a smooth pinball loss with a fixed level of smoothness that appears good enough for many practical cases ... I have already added it to the neural net above, but I need to test it more

 ```
def log_cosh_quantile(alpha):
    def _log_cosh_quantile(y_true, y_pred):
        err = y_pred - y_true
        err = np.where(err < 0, alpha * err, (1 - alpha) * err)
        grad = np.tanh(err)
        hess = 1 / np.cosh(err)**2
        return grad, hess
    return _log_cosh_quantile

 ```